In [1]:
import time
import numpy as np
import tensorflow as tf
import random
from collections import Counter

In [9]:
with open('E:/python_work/handson_ml/datasets/Javasplittedwords.txt','r',encoding='utf-8') as f:
    text = f.read()

In [42]:
text = text.split(' ')

In [43]:
words_count = Counter(text)
words = [w for w in text if words_count[w] > 50]

In [44]:
words_count

Counter({'熟练掌握': 14393,
         'java': 16940,
         '熟悉': 117424,
         'python': 7767,
         'shell': 3494,
         '熟练使用': 20556,
         'git': 1728,
         'svn': 1210,
         '能够': 29684,
         '发现': 4603,
         '问题': 21185,
         '精准': 985,
         '定位问题': 195,
         '快速': 9775,
         '解决问题': 8020,
         'jvm': 844,
         '优化': 35263,
         '经验': 111298,
         '分析': 55814,
         '找出': 311,
         '服务器': 4611,
         '代码': 17103,
         '存在': 953,
         '具有': 53927,
         '服务端': 1594,
         '架构': 14772,
         '理解': 13614,
         '掌握': 13518,
         '理论': 2763,
         '实际操作': 242,
         'dubbo': 472,
         'thrift': 144,
         '工具': 14214,
         'mysql': 14202,
         'redis': 4592,
         '并能': 3691,
         '性能': 11192,
         'kafka': 876,
         'zeromq': 32,
         '使用': 19057,
         '大数据': 10049,
         '高': 9693,
         '并发': 4444,
         '参与': 24440,
         '开源': 6643,


In [46]:
vocab = set(words)
vocab_to_int = {w:c for c ,w in enumerate(vocab)}
int_to_vocab = {c: w for c,w in enumerate(vocab)}

In [47]:
vocab_to_int['word']

3038

In [48]:
print('total words：{}'.format(len(words)))
print('unique words: {}'.format(len(vocab)))

total words：8614953
unique words: 6846


In [49]:
int_words = [vocab_to_int[w] for w in words]

In [50]:
int_words[:10]

[6444, 4257, 4506, 477, 580, 244, 4279, 2437, 5083, 983]

In [51]:
int_to_vocab[1518]

'催收'

In [52]:
t = 1e-5
threshold = 0.8

int_word_counts = Counter(int_words)
total_count = len(int_words)
# 计算单词频率
word_freqs = {w: c/total_count for w, c in int_word_counts.items()}
# 计算被删除的概率
prob_drop = {w: 1 - np.sqrt(t / word_freqs[w]) for w in int_word_counts}
# 对单词进行采样
train_words = [w for w in int_words if prob_drop[w] < threshold]

In [53]:
len(train_words)

2022807

In [20]:
def get_targets(words, idx, window_size=5):
    '''
    获得input word的上下文单词列表
    
    参数
    ---
    words: 单词列表
    idx: input word的索引号
    window_size: 窗口大小
    '''
    target_window = np.random.randint(1, window_size+1)
    # 这里要考虑input word前面单词不够的情况
    start_point = idx - target_window if (idx - target_window) > 0 else 0
    end_point = idx + target_window
    # output words(即窗口中的上下文单词)
    targets = set(words[start_point: idx] + words[idx+1: end_point+1])
    return list(targets)

In [21]:
def get_batches(words, batch_size, window_size=5):
    '''
    构造一个获取batch的生成器
    '''
    n_batches = len(words) // batch_size
    
    # 仅取full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx: idx+batch_size]
        for i in range(len(batch)):
            batch_x = batch[i]
            batch_y = get_targets(batch, i, window_size)
            # 由于一个input word会对应多个output word，因此需要长度统一
            x.extend([batch_x]*len(batch_y))
            y.extend(batch_y)
        yield x, y

In [22]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=[None], name='inputs')
    labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')

In [54]:
vocab_size = len(int_to_vocab)
embedding_size = 200 # 嵌入维度

In [55]:
vocab_size

6846

In [56]:
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform([vocab_size,embedding_size],-1,1))
    embed = tf.nn.embedding_lookup(embedding,inputs)

In [57]:
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal([vocab_size,embedding_size],stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(vocab_size))
    
    loss = tf.nn.sampled_softmax_loss(softmax_w,softmax_b,labels,embed,n_sampled,vocab_size)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [58]:

with train_graph.as_default():
    # 随机挑选一些单词
    ## From Thushan Ganegedara's implementation
    valid_size = 7 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = [
                      vocab_to_int['熟悉'],
                      vocab_to_int['能力'], 
                      vocab_to_int['逻辑思维'],
                      vocab_to_int['了解']]
    
    valid_size = len(valid_examples)
    # 验证单词集
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # 计算每个词向量的模并进行单位化
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    # 查找验证单词的词向量
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    # 计算余弦相似度
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [60]:
epochs = 10 # 迭代轮数
batch_size = 1000 # batch大小
window_size = 10 # 窗口大小

with train_graph.as_default():
    saver = tf.train.Saver() # 文件存储

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        # 
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            # 计算相似的词
            if iteration % 1000 == 0:
                # 计算similarity
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # 取最相似单词的前8个
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to [%s]:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
            
#    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 3.9663 0.3036 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 3.6956 0.2889 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 3.8841 0.2871 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 3.7072 0.3501 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 3.5868 0.3471 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 3.6430 0.3575 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 3.4255 0.3616 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 3.4317 0.3543 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 3.4153 0.3720 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 3.3137 0.3751 sec/batch
Nearest to [熟悉]: 优势, 数据源, 集群, 项目管理, 新领域, 定位问题, 须, 小额贷款,
Nearest to [能力]: 组件化, 通报, 电子相关, 带领团队, 名称, 广告主, 工程师, 之内,
Nearest to [逻辑思维]: 故事, 
正在, 结构设计, 充分考虑, 管理工具, 提高, 受众, 好奇,
Nearest to [了解]: 范, 严格要求, 塑造, 四大, 教育, 出色, 副总经理, 市场策略,
Epoch 1/10 Iteration: 1100 Avg. Training loss: 3.2686 0.3695 sec/batch
Epoc

KeyboardInterrupt: 